In [2]:
import pgeon.policy_graph as PG
from example.environment import SelfDrivingEnvironment
import networkx as nx
from example.discretizer.discretizer_d0 import AVDiscretizer
from example.discretizer.discretizer_d1 import AVDiscretizerD1
from pgeon.NetworkVisualizer import NetworkVisualizer
from collections import Counter


In [3]:
environment = SelfDrivingEnvironment(city='all')
discretizer_configs = {
    'a': {'obj_discretizer': 'binary', 'vel_discretizer': 'binary' },
    'b': {'obj_discretizer': 'multiple', 'vel_discretizer': 'binary'}
    #'c': {'obj_discretizer': 'multiple', 'vel_discretizer': 'multiple'}
    }

default_config = {'obj_discretizer': 'multiple', 'vel_discretizer': 'multiple'}

for discretizer_id in ['0a','0b','1a','1b']:
    config = default_config
    for key in discretizer_configs:
        if key in discretizer_id:
            config = discretizer_configs[key]
            break

    DiscretizerClass = AVDiscretizer if '0' in discretizer_id else AVDiscretizerD1
    discretizer = DiscretizerClass(
            environment,
            vel_discretization=config['vel_discretizer'],
            obj_discretization=config['obj_discretizer'],
            id=discretizer_id
        ) 
    nodes_path = f'example/dataset/data/policy_graphs/PG_trainval_Call_D{discretizer_id}_Wall_Tall_nodes.csv'
    edges_path = f'example/dataset/data/policy_graphs/PG_trainval_Call_D{discretizer_id}_Wall_Tall_edges.csv'
    print(f'DISCRETIZER D{discretizer_id} ')
    pg = PG.PolicyGraph.from_nodes_and_edges(nodes_path, edges_path, environment, discretizer)

    #remove 1-node weakly connected components
    #weakly_connected_components = list(nx.weakly_connected_components(pg))
    #for component in weakly_connected_components:
    #    if len(component) == 1:
    #        pg.remove_node(next(iter(component)))  



    num_nodes = pg.number_of_nodes()
    num_edges = pg.number_of_edges()

    print('Graph Properties:')
    density = num_edges / (num_nodes * (num_nodes - 1))
    sparsity = num_edges / num_nodes
    print(f'density: {density}')
    print(f'sparsity: {sparsity}')
    print(f'|E|: {num_edges}')
    print(f'|V|: {num_nodes}')
    print(f'|V|*(|V|-1): {num_nodes*(num_nodes-1)}')

    cc = list(nx.weakly_connected_components(pg))
    cc_sizes = [len(component) for component in cc]
    
    # Count occurrences of each size
    size_counts = Counter(cc_sizes)
    print(f'Counts for each Weakly Connected Component size: {size_counts}')
    #print('Weakly Connected Components of size 1: ')
    #print([component for component in cc if len(component)==1])

    print()
    nv = NetworkVisualizer(pg, layout='')
    #nv.show(allow_recursion=False, layout='spring')

DISCRETIZER D0a 
Graph Properties:
density: 0.013896007081954992
sparsity: 7.0452755905511815
|E|: 3579
|V|: 508
|V|*(|V|-1): 257556
Counts for each Weakly Connected Component size: Counter({508: 1})

DISCRETIZER D0b 
Graph Properties:
density: 0.007550103913845975
sparsity: 6.259036144578313
|E|: 5195
|V|: 830
|V|*(|V|-1): 688070
Counts for each Weakly Connected Component size: Counter({830: 1})

DISCRETIZER D1a 
Graph Properties:
density: 0.005365317614649165
sparsity: 5.837465564738292
|E|: 6357
|V|: 1089
|V|*(|V|-1): 1184832
Counts for each Weakly Connected Component size: Counter({1089: 1})

DISCRETIZER D1b 
Graph Properties:
density: 0.0029550422004774876
sparsity: 5.0944927536231885
|E|: 8788
|V|: 1725
|V|*(|V|-1): 2973900
Counts for each Weakly Connected Component size: Counter({1725: 1})

